In [1]:
import requests
from pymongo import MongoClient
from datetime import datetime
import time
from neo4j import GraphDatabase

url = 'https://celestrak.org/NORAD/elements/gp.php?GROUP=active&FORMAT=tle'

client = MongoClient('mongodb://localhost:27017/')
mongo_db = client.celestrak
collection = mongo_db.activeSats

neo4j_uri = "bolt://localhost:7687"
neo4j_driver = GraphDatabase.driver(neo4j_uri, auth=("sumit", "abcd12345"))


In [2]:
def check_mongodb_connection():
    try:
        
        mongo_db.command("ping")
        print("MongoDB connection successful.")
    except Exception as e:
        print(f"MongoDB connection failed: {e}")

def check_neo4j_connection():
    try:
        with neo4j_driver.session() as session:
            result = session.run("RETURN 1")
            if result.single()[0] == 1:
                print("Neo4j connection successful.")
    except Exception as e:
        print(f"Neo4j connection failed: {e}")

check_mongodb_connection()
check_neo4j_connection()

MongoDB connection successful.
Neo4j connection successful.


In [3]:
response = requests.get(url)
tle_data = response.text

def parse_tle(tle):
    lines = tle.strip().split('\n')
    if len(lines) == 3:
        name = lines[0].strip()
        line1 = lines[1].strip()
        line2 = lines[2].strip()
        return {
            'name': name,
            'line1': line1,
            'line2': line2,
            'timestamp': int(time.time())
        }
    return None

#(Spliting TLE data into lines)
tle_lines = tle_data.split('\n')
tle_sets = [tle_lines[i:i+3] for i in range(0, len(tle_lines), 3)]

for tle_set in tle_sets:
    tle = '\n'.join(tle_set)
    parsed_tle = parse_tle(tle)
    if parsed_tle:
        
        #Insert if the document does not exist
        collection.update_one(
            {'name': parsed_tle['name']},  
            {'$set': parsed_tle},  
            upsert=True  
        )

print("TLE data has been upserted into MongoDB.")

TLE data has been upserted into MongoDB.


In [4]:
def create_sat_node(tx, satellite):
    query = (
        "MERGE (s:SAT {name: $name}) "
        "ON CREATE SET s.line1 = $line1, s.line2 = $line2, s.timestamp = $timestamp "
        "ON MATCH SET s.line1 = $line1, s.line2 = $line2, s.timestamp = $timestamp "
    )
    tx.run(query, name=satellite['name'], line1=satellite['line1'], line2=satellite['line2'], timestamp=satellite['timestamp'])


satellites = collection.find()


with neo4j_driver.session() as session:
    for satellite in satellites:
        session.execute_write(create_sat_node, satellite)

print("Satellites have been synchronized with Neo4j.")

Satellites have been synchronized with Neo4j.


In [5]:
from datetime import timedelta
from sgp4.api import Satrec, jday


def fetch_satellite_data(name):
    result = collection.find_one({"name": name})
    return result["line1"], result["line2"], result["timestamp"]

def calculate_orbital_path(line1, line2, time_step_minutes=5, duration_hours=24):
    satellite = Satrec.twoline2rv(line1, line2)
    start_time = datetime.utcnow()
    time_step = timedelta(minutes=time_step_minutes)
    duration = timedelta(hours=duration_hours)
    positions = []
    current_time = start_time
    while current_time < start_time + duration:
        jd, fr = jday(current_time.year, current_time.month, current_time.day,
                      current_time.hour, current_time.minute, current_time.second + current_time.microsecond / 1e6)
        e, r, v = satellite.sgp4(jd, fr)
        timestamp_seconds = int(current_time.timestamp())  
        positions.append({"x": r[0], "y": r[1], "z": r[2], "timestamp": timestamp_seconds})
        current_time += time_step
    return positions

#(create nodes in Neo4j for the orbital path of a satellite)
def create_nodes_in_neo4j(tx, satellite_name, positions):
    for i, pos in enumerate(positions):
        query = (
            "MERGE (p:Position {x: $x, y: $y, z: $z, timestamp: $timestamp}) "
            "WITH p "
            "MATCH (s:SAT {name: $satellite_name}) "
            "CREATE (s)-[:HAS_POSITION]->(p)"
        )
        tx.run(query, x=pos["x"], y=pos["y"], z=pos["z"], timestamp=pos["timestamp"], satellite_name=satellite_name)
        
def find_closest_approach(tx, satellite1, satellite2):
    query = (
        "MATCH (s1:SAT {name: $satellite1})-[:HAS_POSITION]->(p1:Position), "
        "(s2:SAT {name: $satellite2})-[:HAS_POSITION]->(p2:Position) "
        "WHERE p1.timestamp = p2.timestamp "  
        "WITH p1, p2, point.distance(point({x: p1.x, y: p1.y, z: p1.z}), point({x: p2.x, y: p2.y, z: p2.z})) AS dist "
        "ORDER BY dist ASC "
        "LIMIT 1 "
        "RETURN p1, p2, dist"
    )
    result = tx.run(query, satellite1=satellite1, satellite2=satellite2)
    return result.single()

In [6]:
satellite1_name = "CALSPHERE 1"
satellite2_name = "CALSPHERE 2"
duration_hours = 24
time_step = 5

#(Fetching TLE data and timestamps from MongoDB)
line1_satellite1, line2_satellite1, timestamp_satellite1 = fetch_satellite_data(satellite1_name)
line1_satellite2, line2_satellite2, timestamp_satellite2 = fetch_satellite_data(satellite2_name)

#(Calculate orbital path for both satellites)
positions_satellite1 = calculate_orbital_path(line1_satellite1, line2_satellite1, time_step, duration_hours)
positions_satellite2 = calculate_orbital_path(line1_satellite2, line2_satellite2, time_step, duration_hours)

print(f'{satellite1_name} : Line 1 ({line1_satellite1}), Line 2 ({line2_satellite1}), timestamp ({timestamp_satellite1})')
print(f'{satellite1_name} Positions : {positions_satellite1}')

print(f'{satellite2_name} : Line 1 ({line1_satellite2}), Line 2 ({line2_satellite2}), timestamp ({timestamp_satellite2})')
print(f'{satellite2_name} Positions : {positions_satellite2}')

with neo4j_driver.session() as session:
    session.execute_write(create_nodes_in_neo4j, satellite1_name, positions_satellite1)
    session.execute_write(create_nodes_in_neo4j, satellite2_name, positions_satellite2)

    closest_approach = session.execute_read(find_closest_approach, satellite1_name, satellite2_name)
    p1 = closest_approach["p1"]
    p2 = closest_approach["p2"]
    dist = closest_approach["dist"]
    print("Nodes of closest approach:")
    print(f'{satellite1_name} and {satellite2_name} are at a distance of {closest_approach["dist"]} km on {datetime.fromtimestamp(p1["timestamp"])}')
    print("Position 1:", p1["x"], p1["y"], p1["z"], p1["timestamp"])
    print("Position 2:", p2["x"], p2["y"], p2["z"], p2["timestamp"])


C:\Users\sumit\AppData\Local\Temp\ipykernel_9436\4026546613.py:11: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  start_time = datetime.utcnow()


CALSPHERE 1 : Line 1 (1 00900U 64063C   24165.21851233  .00000890  00000+0  91981-3 0  9998), Line 2 (2 00900  90.2049  55.2970 0026691 358.2376 122.3869 13.75071480970486), timestamp (1718282144)
CALSPHERE 1 Positions : [{'x': -3519.1779245912116, 'y': -5057.480490191191, 'z': -4065.195037151775, 'timestamp': 1718275249}, {'x': -2687.841042029513, 'y': -3846.4855567832096, 'z': -5693.794548279295, 'timestamp': 1718275549}, {'x': -1618.778837308255, 'y': -2295.289243027976, 'z': -6817.596456815382, 'timestamp': 1718275849}, {'x': -406.50330701972734, 'y': -541.0291589946121, 'z': -7336.713484155499, 'timestamp': 1718276149}, {'x': 841.7547199550336, 'y': 1261.1170744954507, 'z': -7204.161228730097, 'timestamp': 1718276449}, {'x': 2015.1816874146755, 'y': 2951.154468094901, 'z': -6430.103560700342, 'timestamp': 1718276749}, {'x': 3008.9254626930606, 'y': 4378.054396817792, 'z': -5081.5623360236095, 'timestamp': 1718277049}, {'x': 3733.4284329608486, 'y': 5413.203872516204, 'z': -3277.38

In [7]:
import plotly.graph_objects as go
import pytz

def format_timestamp_to_cet(timestamp):
    dt_utc = datetime.utcfromtimestamp(timestamp)
    dt_utc = pytz.utc.localize(dt_utc)
    dt_cet = dt_utc.astimezone(pytz.timezone('CET'))
    return dt_cet.strftime('%Y-%m-%d %H:%M:%S')


x1 = [pos['x'] for pos in positions_satellite1]
y1 = [pos['y'] for pos in positions_satellite1]
z1 = [pos['z'] for pos in positions_satellite1]
timestamps1 = [format_timestamp_to_cet(pos['timestamp']) for pos in positions_satellite1]


x2 = [pos['x'] for pos in positions_satellite2]
y2 = [pos['y'] for pos in positions_satellite2]
z2 = [pos['z'] for pos in positions_satellite2]
timestamps2 = [format_timestamp_to_cet(pos['timestamp']) for pos in positions_satellite2]

fig = go.Figure()


fig.add_trace(go.Scatter3d(
    x=x1,
    y=y1,
    z=z1,
    mode='markers+lines',
    marker=dict(
        size=4,
        color='blue',
    ),
    line=dict(
        color='blue',
        width=2
    ),
    name=satellite1_name,
    text=timestamps1
))


fig.add_trace(go.Scatter3d(
    x=x2,
    y=y2,
    z=z2,
    mode='markers+lines',
    marker=dict(
        size=4,
        color='red',
    ),
    line=dict(
        color='red',
        width=2
    ),
    name=satellite2_name,
    text=timestamps2
))


fig.update_layout(
    title=f'Orbital Paths of {satellite1_name} and {satellite2_name}',
    scene=dict(
        xaxis_title='X (km)',
        yaxis_title='Y (km)',
        zaxis_title='Z (km)'
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)


fig.show()

C:\Users\sumit\AppData\Local\Temp\ipykernel_9436\1948049206.py:5: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt_utc = datetime.utcfromtimestamp(timestamp)
